In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
import pandas as pd
import numpy as np
from collections import Counter
import re
from nltk.tokenize import word_tokenize
from tqdm import tqdm
import os

In [27]:
project_path = 'GAN-Output'
if not os.path.exists(project_path):
    os.makedirs(project_path)
print(f"Project directory created at {project_path}")

# Define the CSV file path
csv_path = r'Urdu_Dataset\urdu_batch_1.csv'  # Update this with the actual path to your CSV file
data_path = r'GAN-Output\urdu_text.txt'  # Update this with the desired path to save the text file


Project directory created at GAN-Output


In [28]:
# Download required NLTK data
nltk.download('punkt')

# First, ensure we have the device set up
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\faiza\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
# Function to preprocess and combine text data from the CSV\
def load_and_process_data(csv_path):
    try:
        # Load the CSV file into a DataFrame
        df = pd.read_csv(csv_path)

        # Ensure the relevant column containing the text data is named appropriately
        if 'News Text' not in df.columns:
            raise ValueError("Expected column 'News Text' not found in the CSV file.")

        # Preprocess and combine all text data into a single string
        all_text = ' '.join(df['News Text'].dropna().tolist())

        return all_text

    except Exception as e:
        print(f"Error loading or processing the CSV file: {str(e)}")
        return None

# Load and process data from the CSV file
article_text = load_and_process_data(csv_path)

# Save the processed data to a text file
if article_text:
    try:
        with open(data_path, 'w', encoding='utf-8') as file:
            file.write(article_text)
        print(f"Data saved successfully at: {data_path}")
    except Exception as e:
        print(f"Error saving the data to file: {str(e)}")
else:
    print("No data to save.")

Data saved successfully at: GAN-Output\urdu_text.txt


In [30]:
def clean_text(text):
    # Remove non-Urdu characters while keeping necessary punctuation
    urdu_pattern = r'[\u0600-\u06FF\s\.\،\؟]+'
    cleaned = re.findall(urdu_pattern, text)
    cleaned_text = ' '.join(cleaned)

    # Remove extra whitespace
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    # Remove very short lines
    cleaned_text = '\n'.join(line for line in cleaned_text.split('\n')
                            if len(line.strip()) > 10)

    return cleaned_text

# Load and clean text
with open(data_path, 'r', encoding='utf-8') as file:
    text = file.read()

cleaned_text = clean_text(text)
tokens = word_tokenize(cleaned_text)

# Create vocabulary
special_tokens = ['<PAD>', '<UNK>', '<START>', '<END>']
word2id = {token: idx for idx, token in enumerate(special_tokens)}

# Count word frequencies
vocab = Counter(tokens)
min_freq = 2  # Minimum frequency threshold

# Add frequent words to vocabulary
idx = len(special_tokens)
for word, count in vocab.most_common():
    if count >= min_freq:
        word2id[word] = idx
        idx += 1

id2word = {idx: word for word, idx in word2id.items()}
vocab_size = len(word2id)

print(f"Total tokens: {len(tokens)}")
print(f"Vocabulary size: {vocab_size}")

Total tokens: 7648
Vocabulary size: 828


In [31]:
class UrduDataset(Dataset):
    def __init__(self, tokens, word2id, seq_length):
        self.tokens = tokens
        self.word2id = word2id
        self.seq_length = seq_length
        self.sequences = self._create_sequences()

        if len(self.sequences) == 0:
            raise ValueError("No sequences were created. Check your data and sequence length.")

    def _create_sequences(self):
        sequences = []
        for i in range(0, len(self.tokens) - self.seq_length, self.seq_length):
            seq = self.tokens[i:i + self.seq_length + 1]
            if len(seq) == self.seq_length + 1:
                seq_ids = [self.word2id.get(word, self.word2id['<UNK>'])
                          for word in seq]
                sequences.append(seq_ids)
        return sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        return (torch.tensor(sequence[:-1], dtype=torch.long),
                torch.tensor(sequence[1:], dtype=torch.long))

# Create dataset and dataloader
seq_length = 10  # Adjusted for small dataset
batch_size = 8   # Adjusted for small dataset

dataset = UrduDataset(tokens, word2id, seq_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

print(f"Dataset size: {len(dataset)}")
print(f"Number of batches: {len(dataloader)}")

Dataset size: 764
Number of batches: 95


In [32]:
class Generator(nn.Module):
    def __init__(self, vocab_size, embedding_dim=256, hidden_dim=512, num_layers=2, dropout=0.2):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        # Core layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )

        # Attention mechanism
        self.attention = nn.MultiheadAttention(hidden_dim * 2, num_heads=4)

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim * 4)
        self.fc2 = nn.Linear(hidden_dim * 4, hidden_dim * 2)
        self.output_layer = nn.Linear(hidden_dim * 2, vocab_size)

        # Regularization
        self.dropout = nn.Dropout(dropout)
        self.layer_norm1 = nn.LayerNorm(hidden_dim * 2)
        self.layer_norm2 = nn.LayerNorm(hidden_dim * 2)

        # Activation
        self.gelu = nn.GELU()

    def forward(self, x):
        # Embedding
        embedded = self.dropout(self.embedding(x))

        # LSTM
        lstm_out, _ = self.lstm(embedded)

        # Self-attention
        attn_output, _ = self.attention(
            lstm_out.transpose(0, 1),
            lstm_out.transpose(0, 1),
            lstm_out.transpose(0, 1)
        )
        attn_output = attn_output.transpose(0, 1)

        # First residual + norm
        output = self.layer_norm1(lstm_out + attn_output)

        # Fully connected with residual
        residual = output
        output = self.fc1(output)
        output = self.gelu(output)
        output = self.dropout(output)
        output = self.fc2(output)

        # Second residual + norm
        output = self.layer_norm2(output + residual)

        # Output
        logits = self.output_layer(output)

        return logits, None

class Generator(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=2, dropout=0.5):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim

        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM layers
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )

        # Attention mechanism
        self.attention = nn.MultiheadAttention(hidden_dim * 2, num_heads=8)

        # Fully connected layers with increased capacity
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim * 4)
        self.fc2 = nn.Linear(hidden_dim * 4, hidden_dim * 2)
        self.output_layer = nn.Linear(hidden_dim * 2, vocab_size)

        # Regularization and normalization
        self.dropout = nn.Dropout(dropout)
        self.layer_norm1 = nn.LayerNorm(hidden_dim * 2)
        self.layer_norm2 = nn.LayerNorm(hidden_dim * 2)

        # Activation functions
        self.gelu = nn.GELU()

    def forward(self, x):
        # Embedding
        embedded = self.dropout(self.embedding(x))

        # LSTM processing
        lstm_out, _ = self.lstm(embedded)

        # Self-attention mechanism
        attn_output, _ = self.attention(
            lstm_out.transpose(0, 1),
            lstm_out.transpose(0, 1),
            lstm_out.transpose(0, 1)
        )
        attn_output = attn_output.transpose(0, 1)

        # First residual connection and layer norm
        output = self.layer_norm1(lstm_out + attn_output)

        # Fully connected layers with residual connection
        residual = output
        output = self.fc1(output)
        output = self.gelu(output)
        output = self.dropout(output)
        output = self.fc2(output)

        # Second residual connection and layer norm
        output = self.layer_norm2(output + residual)

        # Final output layer
        logits = self.output_layer(output)

        return logits, None

class Discriminator(nn.Module):
    def __init__(self, vocab_size, embedding_dim=256, hidden_dim=512, num_layers=2, dropout=0.2):
        super().__init__()

        # Core layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=True
        )

        # Attention
        self.attention = nn.MultiheadAttention(hidden_dim * 2, num_heads=4)

        # Fully connected
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim * 4)
        self.fc2 = nn.Linear(hidden_dim * 4, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, 1)

        # Regularization
        self.dropout = nn.Dropout(dropout)
        self.layer_norm1 = nn.LayerNorm(hidden_dim * 2)
        self.layer_norm2 = nn.LayerNorm(hidden_dim)

        # Activation
        self.gelu = nn.GELU()

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))

        # LSTM
        lstm_out, _ = self.lstm(embedded)

        # Self-attention
        attn_output, _ = self.attention(
            lstm_out.transpose(0, 1),
            lstm_out.transpose(0, 1),
            lstm_out.transpose(0, 1)
        )
        attn_output = attn_output.transpose(0, 1)

        # Residual + norm
        output = self.layer_norm1(lstm_out + attn_output)

        # Global average pooling
        output = torch.mean(output, dim=1)

        # Fully connected layers
        output = self.fc1(output)
        output = self.gelu(output)
        output = self.dropout(output)

        output = self.fc2(output)
        output = self.layer_norm2(output)
        output = self.gelu(output)
        output = self.dropout(output)

        # Final classification
        output = self.fc3(output)
        return torch.sigmoid(output)

In [33]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
        self.criterion = nn.KLDivLoss(reduction='batchmean')

    def forward(self, pred, target):
        smoothed_target = torch.zeros_like(pred)
        smoothed_target.fill_(self.smoothing / (pred.size(-1) - 1))
        smoothed_target.scatter_(1, target.unsqueeze(1), 1 - self.smoothing)
        return self.criterion(pred.log_softmax(dim=-1), smoothed_target)

class EarlyStopping:
    def __init__(self, patience=40, min_delta=0.001):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            return False

        if val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                return True
        else:
            self.best_loss = val_loss
            self.counter = 0
        return False

def add_noise_to_inputs(tensor, noise_factor=0.1):
    noise = torch.randn_like(tensor) * noise_factor
    return tensor + noise

def get_warmup_schedule(optimizer, num_warmup_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return 1.0
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
# Model hyperparameters
embedding_dim = 256
hidden_dim = 512
num_layers = 2
dropout = 0.2

# Initialize models
generator = Generator(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    dropout=dropout
).to(device)

discriminator = Discriminator(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    dropout=dropout
).to(device)

# Optimizers
g_optimizer = torch.optim.AdamW(
    generator.parameters(),
    lr=0.0001,
    betas=(0.5, 0.999),
    weight_decay=0.001
)

d_optimizer = torch.optim.AdamW(
    discriminator.parameters(),
    lr=0.0001,
    betas=(0.5, 0.999),
    weight_decay=0.001
)

# Schedulers
g_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    g_optimizer,
    mode='min',
    factor=0.7,
    patience=10,
    
)

d_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    d_optimizer,
    mode='min',
    factor=0.7,
    patience=10,

)

# Loss functions
adversarial_criterion = nn.BCELoss()
content_criterion = LabelSmoothingLoss(smoothing=0.1)

In [39]:
def compute_gradient_penalty(discriminator, real_samples, fake_samples, device):
    batch_size = real_samples.size(0)
    seq_len = real_samples.size(1)

    # Create random weight for interpolation
    alpha = torch.rand(batch_size, 1).expand(-1, seq_len).to(device)

    # Convert to float for interpolation
    real_float = real_samples.float()
    fake_float = fake_samples.float()

    # Interpolate between real and fake samples
    interpolates = (alpha * real_float + ((1 - alpha) * fake_float))

    # Create embeddings for interpolated samples
    with torch.no_grad():
        discrete_interpolates = torch.clamp(interpolates.round(), min=0, max=vocab_size-1).long()
        embedded_interpolates = discriminator.embedding(discrete_interpolates)

    embedded_interpolates.requires_grad_(True)

    # Disable CuDNN for gradient computation
    with torch.backends.cudnn.flags(enabled=False):
        # Pass embedded interpolates through the discriminator components
        embedded = discriminator.dropout(embedded_interpolates)
        output, _ = discriminator.lstm(embedded)
        output = torch.mean(output, dim=1)
        output = discriminator.fc1(output)
        output = F.relu(output)
        output = discriminator.dropout(output)
        d_interpolates = discriminator.fc2(output)
        d_interpolates = torch.sigmoid(d_interpolates)

    # Calculate gradients
    grad_outputs = torch.ones_like(d_interpolates).to(device)
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=embedded_interpolates,
        grad_outputs=grad_outputs,
        create_graph=True,
        retain_graph=True,
        only_inputs=True
    )[0]

    # Calculate gradient penalty
    gradients = gradients.reshape(batch_size, -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()

    return gradient_penalty

    # Define optimizers and loss functions
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

adversarial_criterion = nn.BCELoss()
content_criterion = nn.CrossEntropyLoss(ignore_index=word2id['<PAD>'])

class EarlyStopping:
    def __init__(self, patience=15, min_delta=0.001):  # Increased patience and added min_delta
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.val_loss_min = float('inf')

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            return False

        if val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                return True
        else:
            self.best_loss = val_loss
            self.counter = 0
        return False

def train_gan(generator, discriminator, dataloader, num_epochs, device, gradient_accumulation_steps=4):
    early_stopping = EarlyStopping(patience=40, min_delta=0.001)
    torch.cuda.empty_cache()

    # Warmup
    warmup_steps = 100
    warmup_scheduler_g = get_warmup_schedule(g_optimizer, warmup_steps)
    warmup_scheduler_d = get_warmup_schedule(d_optimizer, warmup_steps)

    # Track best losses
    best_g_loss = float('inf')
    best_d_loss = float('inf')

    epoch_pbar = tqdm(range(num_epochs), desc='Training Progress')
    print("Starting training... inside train gan")
    for epoch in epoch_pbar:
        total_d_loss = 0
        total_g_loss = 0
        batch_count = 0
        print("Inside loop now")
        generator.train()
        discriminator.train()
        print("After functions")
        batch_pbar = tqdm(dataloader, leave=False, desc=f'Epoch {epoch}')

        for i, (real_seq, target_seq) in enumerate(batch_pbar):
            batch_size = real_seq.size(0)
            real_seq = real_seq.to(device)
            target_seq = target_seq.to(device)
            print("Inside inner loop")
            # Train Discriminator
            discriminator.zero_grad()

            # Real samples with noise
            noisy_real_seq = add_noise_to_inputs(real_seq.float(), noise_factor=0.1).long()
            real_outputs = discriminator(noisy_real_seq)
            real_labels = torch.ones(batch_size, 1).to(device)
            d_loss_real = adversarial_criterion(real_outputs, real_labels)

            # Fake samples with noise
            with torch.no_grad():
                noise = torch.randint(0, vocab_size, (batch_size, real_seq.size(1)), device=device)
                fake_seq, _ = generator(noise)
                fake_tokens = fake_seq.argmax(dim=-1)

            noisy_fake_tokens = add_noise_to_inputs(fake_tokens.float(), noise_factor=0.1).long()
            fake_outputs = discriminator(noisy_fake_tokens)
            fake_labels = torch.zeros(batch_size, 1).to(device)
            d_loss_fake = adversarial_criterion(fake_outputs, fake_labels)

            # Total discriminator loss with gradient accumulation
            d_loss = (d_loss_real + d_loss_fake) / gradient_accumulation_steps
            d_loss.backward()

            if (i + 1) % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(discriminator.parameters(), max_norm=1.0)
                d_optimizer.step()
                discriminator.zero_grad()

            # Train Generator
            generator.zero_grad()

            # Generate new fake samples
            fake_seq, _ = generator(noise)
            fake_tokens = fake_seq.argmax(dim=-1)
            fake_outputs = discriminator(fake_tokens)

            # Generator losses with gradient accumulation
            g_loss_adv = adversarial_criterion(fake_outputs, real_labels)
            g_loss_content = content_criterion(fake_seq.transpose(1, 2), target_seq)
            g_loss = (0.5 * g_loss_adv + 0.5 * g_loss_content) / gradient_accumulation_steps

            g_loss.backward()

            if (i + 1) % gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(generator.parameters(), max_norm=1.0)
                g_optimizer.step()
                generator.zero_grad()

            # Update tracking
            total_d_loss += d_loss.item() * gradient_accumulation_steps
            total_g_loss += g_loss.item() * gradient_accumulation_steps
            batch_count += 1

            batch_pbar.set_postfix({
                'D_loss': f'{d_loss.item():.4f}',
                'G_loss': f'{g_loss.item():.4f}'
            })
        print("After inner loop")
        # Calculate average losses
        avg_d_loss = total_d_loss / batch_count
        avg_g_loss = total_g_loss / batch_count

        # Update learning rates
        if epoch < warmup_steps:
            warmup_scheduler_g.step()
            warmup_scheduler_d.step()
        else:
            g_scheduler.step(avg_g_loss)
            d_scheduler.step(avg_d_loss)

        # Save best models
        if avg_g_loss < best_g_loss:
            best_g_loss = avg_g_loss
            torch.save(generator.state_dict(), 'best_generator.pt')
        if avg_d_loss < best_d_loss:
            best_d_loss = avg_d_loss
            torch.save(discriminator.state_dict(), 'best_discriminator.pt')

        # Early stopping check
        if early_stopping(avg_g_loss):
            print("\nEarly stopping triggered!")
            break

    return generator, discriminator

In [40]:
# Training execution with adjusted parameters
try:
    print("Starting training...")
    print(f"Vocabulary size: {vocab_size}")

    # Adjusted training parameters
    num_epochs = 20  # Increased from 100
    batch_size = 25   # Increased from 8

    # Create new dataloader with adjusted batch size
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    print("1st Step")
    # Initialize early stopping with new parameters
    early_stopping = EarlyStopping(patience=15, min_delta=0.001)
    print("2nd Step")
    # Train the models
    generator, discriminator = train_gan(generator, discriminator, dataloader, num_epochs, device)
    print("Training completed successfully!")
    print("Models saved as 'best_generator.pt' and 'best_discriminator.pt'")
    # Save trained model
    torch.save({
        'generator_state': generator.state_dict(),
        'discriminator_state': discriminator.state_dict(),
        'g_optimizer_state': g_optimizer.state_dict(),
        'd_optimizer_state': d_optimizer.state_dict(),
    }, os.path.join(project_path, 'trained_model.pt'))

except Exception as e:
    print(f"Training failed: {str(e)}")
    raise

Inside inner loop


Inside inner loop


Inside inner loop


Inside inner loop


Inside inner loop


...

Inside inner loop


Inside inner loop


Inside inner loop


Training Progress: 100%|██████████| 20/20 [28:25<00:00, 85.27s/it]


After inner loop
EarlyStopping counter: 15 out of 40
Training completed successfully!
Models saved as 'best_generator.pt' and 'best_discriminator.pt'


In [41]:
def generate_text(generator, word2id, id2word, seq_length=10, num_samples=5, temperature=1.0, device='cuda'):
    generator.eval()
    generated_texts = []

    with torch.no_grad():
        for _ in range(num_samples):
            current_sequence = [word2id['<START>']]

            for _ in range(seq_length):
                sequence_tensor = torch.LongTensor([current_sequence]).to(device)
                output, _ = generator(sequence_tensor)

                next_token_logits = output[0, -1, :] / temperature
                next_token_probs = F.softmax(next_token_logits, dim=0)
                next_token_id = torch.multinomial(next_token_probs, 1).item()

                current_sequence.append(next_token_id)

                if next_token_id == word2id['<END>']:
                    break

            generated_words = [id2word[id] for id in current_sequence
                             if id not in [word2id['<START>'], word2id['<END>'], word2id['<PAD>']]]

            generated_text = ' '.join(generated_words)
            generated_texts.append(generated_text)

    return generated_texts

# Generate samples with different temperatures
print("Generating text samples with different temperatures...")
temperatures = [0.5, 0.7, 1.0]

for temp in temperatures:
    print(f"\nTemperature: {temp}")
    print("-" * 50)

    generated_samples = generate_text(
        generator=generator,
        word2id=word2id,
        id2word=id2word,
        seq_length=20,
        num_samples=3,
        temperature=temp,
        device=device
    )

    for i, text in enumerate(generated_samples, 1):
        print(f"\nSample {i}:")
        print(text)

Generating text samples with different temperatures...

Temperature: 0.5
--------------------------------------------------

Sample 1:
<UNK> <UNK> توقع کا <UNK> <UNK> <UNK> <UNK> <UNK> کی کا <UNK> نے <UNK> کے <UNK> کی <UNK> کی کے

Sample 2:
کا <UNK> <UNK> <UNK> <UNK> <UNK> میں <UNK> ادائیگی <UNK> کی <UNK> <UNK> کے کی <UNK> سے <UNK> <UNK> <UNK>

Sample 3:
<UNK> کے <UNK> <UNK> کی میں <UNK> میں ارب کے کے <UNK> <UNK> <UNK> کی <UNK> <UNK> کی <UNK> <UNK>

Temperature: 0.7
--------------------------------------------------

Sample 1:
کا <UNK> لیے <UNK> <UNK> <UNK> کر میں کی میں <UNK> <UNK> کی میں سے بعد <UNK> کی سے کہا

Sample 2:
سے <UNK> کی <UNK> کی کی میں <UNK> نے پر کی <UNK> کی میں نے نے <UNK> <UNK> میں <UNK>

Sample 3:
<UNK> عثمانی کی میں میں ڈالر <UNK> میں کے <UNK> <UNK> بینک <UNK> نے ڈی کی کے فیصد <UNK> <UNK>

Temperature: 1.0
--------------------------------------------------

Sample 1:
مزید پاور <UNK> کی کی متعارف کی سے میں <UNK> <UNK> کی کہ کے تھا سطح <UNK> فائل فنڈز سے

Sample 2:
یع

In [42]:
def generate_interactive():
    print("\nInteractive Text Generation")
    print("Enter parameters (press Enter for defaults)")

    try:
        seq_length = int(input("Sequence length (default: 20): ") or 20)
        num_samples = int(input("Number of samples (default: 1): ") or 1)
        temperature = float(input("Temperature (0.1-2.0, default: 1.0): ") or 1.0)

        generated_samples = generate_text(
            generator=generator,
            word2id=word2id,
            id2word=id2word,
            seq_length=seq_length,
            num_samples=num_samples,
            temperature=temperature,
            device=device
        )

        print("\nGenerated Text:")
        print("-" * 50)
        for i, text in enumerate(generated_samples, 1):
            print(f"\nSample {i}:")
            print(text)

    except ValueError as e:
        print(f"Error: Invalid input - {e}")
        return

# Run interactive generation
while True:
    choice = input("\nWould you like to generate text? (y/n): ").lower()
    if choice != 'y':
        break
    generate_interactive()


Interactive Text Generation
Enter parameters (press Enter for defaults)

Generated Text:
--------------------------------------------------

Sample 1:
کے کی میں کے <UNK> کے کی <UNK> <UNK> <UNK> <UNK> کی <UNK> <UNK> میں جو <UNK> کی <UNK> <UNK> کے <UNK> <UNK> میں کی میں کی <UNK> کے کے سال کے <UNK> <UNK> کی <UNK> کے کی کے <UNK> کا کے <UNK> <UNK> کی کے کے سے <UNK> <UNK> <UNK> <UNK> کے <UNK> <UNK> <UNK> <UNK> میں کی <UNK> کے <UNK> کے کے کے کی کی میں <UNK> <UNK> کی <UNK> <UNK> <UNK> کی <UNK> کی <UNK> <UNK> <UNK> افراد <UNK> کی <UNK> کی <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> کی کے سے <UNK> <UNK> <UNK> کی میں کے

Sample 2:
کی میں کی <UNK> <UNK> <UNK> <UNK> کی <UNK> <UNK> <UNK> کی <UNK> <UNK> <UNK> کے کی <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> <UNK> کے ریٹیل کے کے کی کے <UNK> <UNK> میں <UNK> کے کی کے <UNK> <UNK> کی <UNK> میں کی <UNK> کی <UNK> کے کی <UNK> کی <UNK> میں <UNK> <UNK> <UNK> <UNK> کی کے کی کے <UNK> کی <UNK> <UNK> کے کی کے <UNK> <UNK> میں <UNK> کی سے میں <UNK> <UNK> سی میں <UNK> <UNK> کے <UN